# Scratch - SMO
---

In [ ]:
import numpy as np

class SVC:
    def __init__(self, C=1.0, tol=1e-3, max_passes=100, gamma=0.1):
        """
        Inisialisasi parameter untuk Support Vector Classifier.

        Parameters:
        -----------
        C : float, default=1.0
            Parameter regularisasi. Nilai lebih besar akan meningkatkan kesalahan
            margin yang lebih kecil, tetapi dapat menyebabkan overfitting.

        tol : float, default=1e-3
            Toleransi yang digunakan untuk memutuskan kapan algoritma selesai.

        max_passes : int, default=100
            Jumlah maksimal iterasi yang dilakukan untuk memverifikasi konvergensi.

        gamma : float, default=0.1
            Parameter untuk kernel Radial Basis Function (RBF).
        """
        self.C = C  # control threshold for margin
        self.tol = tol  # tolerance threshold
        self.max_passes = max_passes  # number of iterations
        self.gamma = gamma # parameter for RBF kernel

    def fit(self, X, y):
        """Fitting model"""
        # Convert to numpy arrays
        X = np.array(X).copy()
        y = np.array(y).copy()

        n_samples, n_features = X.shape

        # Initialize parameters
        self._initialize_parameters(X)
        passes = 0  # Iteration counter

        while passes < self.max_passes:
            num_changed_alphas = 0

            for i in range(n_samples):
                # Calculate error
                E_i = self._calculate_E(X[i], y[i], X, y)

                # Check conditions for optimization
                cond_1 = (y[i] * E_i < -self.tol) and (self.alpha[i] < self.C)
                cond_2 = (y[i] * E_i > self.tol) and (self.alpha[i] > 0)

                if cond_1 or cond_2:
                    # Select a second sample randomly
                    j = np.random.randint(0, n_samples)

                    # Extract data
                    X_j, y_j, a_j = X[j], y[j], self.alpha[j]

                    # Calculate error for the second sample
                    E_j = self._calculate_E(X_j, y_j, X, y)

                    # Save old alphas
                    old_a_i, old_a_j = np.copy(self.alpha[i]), np.copy(self.alpha[j])

                    # Boundary adjustments for alpha
                    L, H = self._compute_boundary(y[i], y_j, self.alpha[i], a_j)

                    if L == H:  # No change, move to next iteration
                        continue

                    # Compute eta
                    eta = 2 * self._kernel(X[i], X_j) - self._kernel(X[i], X[i]) - self._kernel(X_j, X_j)

                    if eta >= 0:  # Skip if eta is not negative
                        continue

                    # Update alpha_j
                    a_j_unclipped = old_a_j - (y_j * (E_i - E_j)) / eta

                    # Clip alpha_j to the range [L, H]
                    a_j_new = np.clip(a_j_unclipped, L, H)

                    if np.abs(a_j_new - old_a_j) < self.tol:
                        continue  # No significant change, move to next iteration

                    # Update alpha_i
                    a_i_new = self.alpha[i] + (y[i] * y_j) * (old_a_j - a_j_new)

                    # Compute b_1 and b_2
                    b_old = self.intercept_

                    # Calculate b_1
                    b_1 = b_old - E_i - y[i] * (a_i_new - old_a_i) * np.dot(X[i], X[i]) \
                          - y_j * (a_j_new - old_a_j) * np.dot(X[i], X_j)

                    # Calculate b_2
                    b_2 = b_old - E_j - y_j * (a_j_new - old_a_j) * np.dot(X_j, X_j) \
                          - y[i] * (a_i_new - old_a_i) * np.dot(X_j, X[i])

                    # Set b to b_1, b_2, or average
                    if 0 < a_i_new < self.C:
                        b_new = b_1
                    elif 0 < a_j_new < self.C:
                        b_new = b_2
                    else:
                        b_new = (b_1 + b_2) / 2

                    # Update alpha and intercept
                    self.alpha[i], self.alpha[j] = a_i_new, a_j_new
                    self.intercept_ = b_new
                    self._compute_coef(X, y)

                    # Increment number of changed alphas
                    num_changed_alphas += 1

            # Stopping condition
            if num_changed_alphas == 0:
                passes += 1
            else:
                passes = 0

    def predict(self, X):
        """Make predictions"""
        return np.sign(np.dot(X, self.coef_) + self.intercept_).astype(int)

    def _compute_coef(self, X, y):
        """Compute coefficients"""
        self.coef_ = np.dot((self.alpha * y).T, X)

    def _compute_boundary(self, y_i, y_j, a_i, a_j):
        """Upper and lower boundary for a_j"""
        if y_i != y_j:
            L = max(0, a_j - a_i)
            H = min(self.C, self.C + a_j - a_i)
        else:
            L = max(0, a_i + a_j - self.C)
            H = min(self.C, a_i + a_j)
        return L, H

    def _initialize_parameters(self, X):
        """Initialize parameters before training"""
        n_samples, n_features = X.shape
        self.alpha = np.zeros(n_samples)  # Weight for each data point
        self.coef_ = np.zeros(n_features)  # Contribution of features for output
        self.intercept_ = 0.0  # Hyperplane intercept

    def _calculate_E(self, X_star, y_star, X, y):
        """
        Calculate the error (E) between the predicted and true labels.

        Parameters:
        -----------
        X_star : array-like, shape (n_samples, n_features)
            Samples to be predicted.

        y_star : array-like, shape (n_samples,)
            True labels for `X_star`.

        X : array-like, shape (n_samples, n_features)
            Training data (not used in error calculation).

        y : array-like, shape (n_samples,)
            True labels for training data (not used in error calculation).

        Returns:
        --------
        E : array-like, shape (n_samples,)
            Prediction error (predicted - true labels).
        """

        # _calculate_F
        _calculate_F = np.dot(self.alpha, np.dot(X, X_star.T)) + self.intercept_

        # Calculate error
        f = _calculate_F  # Get predictions
        E = f - y_star  # Calculate error
        return E

    def _kernel(self, x1, x2):
        """Compute RBF kernel (Gaussian Kernel)"""
        return np.exp(-self.gamma * np.linalg.norm(x1 - x2) ** 2)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Load Iris dataset
data = load_iris()
X = data.data
y = data.target

# Hanya menggunakan dua kelas untuk klasifikasi biner
X = X[y != 2]
y = y[y != 2]

# Bagi data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Scratch
---

In [ ]:
# Inisialisasi dan latih model
model = SVC(C=1.0, tol=1e-3, gamma=0.1)
model.fit(X_train, y_train)

# Prediksi dan evaluasi akurasi
y_pred = model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)

print(f"Hasil Prediksi: {y_pred}")
print(f"Akurasi: {accuracy * 100:.2f}%")

Hasil Prediksi: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]
Akurasi: 47.50%
